In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import lxml
from bs4 import NavigableString
import re

# Must gather rating data for all the games without data in the ratings_df in the exploratory analysis notebook.
# Must gather positive, neutral, and negative ratings numbers for both users and critics as well as the
# metascore and userscore

no_rating_df = pd.read_csv("../Resources/games_to_get_ratings.csv")

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [22]:
browser = Browser("chrome", executable_path="/usr/local/bin/chromedriver", headless=True)

In [4]:
games_df = pd.read_csv("../Resources/vgsales.csv.zip")
list_of_games = games_df["Name"].values.tolist()

In [5]:
## Navigating to the correct page via the search bar using splinter is too slow. I will rewrite the code to manually 
# write in the url using string interpolation. 
# I will need to interpolate the correct console name as well into the url. Metacritics url structure is:
# "https://www.metacritic.com/game/{console}/{game}"

In [6]:
# must change game names into the proper format to be interpolated into the URL.

game_conversions= {
    "PSV":"playstation-vita",
    "DC":"dreamcast",
    "WiiU":"wii-u",
    "GC":"gamecube",
    "N64":"nintendo-64",
    "XB":"xbox",
    "PSP":"psp",
    "PS4":"playstation-4",
    "PS": "playstation",
    "Wii":"wii",
    "PS3": "playstation-3",
    "PS2":"playstation-2",
    "GBA":"game-boy-advance",
    "DS":"ds",
    "XB":"xbox",
    "X360":"xbox-360"
}

def convert_to_name(platform_abb):
    if platform_abb in game_conversions:
        full_platform= game_conversions[platform_abb]
    else:
        full_platform="other"
    return full_platform
    

In [7]:
# This actually shows that the vast majority of our data belongs to the consoles listed on metacritic, these "legacy consoles"
# Get rid of those games which fall into the "other" category under full_platform_name
games_df["full_platform_name"]= games_df["Platform"].map(convert_to_name)
games_df = games_df.loc[games_df["full_platform_name"]!="other"]


In [8]:
games_df["Name"].tolist()

['Wii Sports',
 'Mario Kart Wii',
 'Wii Sports Resort',
 'New Super Mario Bros.',
 'Wii Play',
 'New Super Mario Bros. Wii',
 'Nintendogs',
 'Mario Kart DS',
 'Wii Fit',
 'Wii Fit Plus',
 'Kinect Adventures!',
 'Grand Theft Auto V',
 'Grand Theft Auto: San Andreas',
 'Brain Age: Train Your Brain in Minutes a Day',
 'Pokemon Diamond/Pokemon Pearl',
 'Grand Theft Auto V',
 'Grand Theft Auto: Vice City',
 'Pokemon Ruby/Pokemon Sapphire',
 'Pokemon Black/Pokemon White',
 'Brain Age 2: More Training in Minutes a Day',
 'Gran Turismo 3: A-Spec',
 'Call of Duty: Modern Warfare 3',
 'Call of Duty: Black Ops',
 'Call of Duty: Black Ops 3',
 'Call of Duty: Black Ops II',
 'Call of Duty: Black Ops II',
 'Call of Duty: Modern Warfare 2',
 'Call of Duty: Modern Warfare 3',
 'Grand Theft Auto III',
 'Super Smash Bros. Brawl',
 'Call of Duty: Black Ops',
 'Animal Crossing: Wild World',
 'Halo 3',
 'Grand Theft Auto V',
 'Pokemon HeartGold/Pokemon SoulSilver',
 'Super Mario 64',
 'Gran Turismo 4',
 'S

In [9]:
# This worked pretty well, but I will alter the code further see if I can increase my scraping success to fail ratio

In [ ]:
game_side_details = []
problem_games = []
abberant_side_detail_games = []
game_info_list = []
counter = 1

# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))


for platform, name in zipped_list:
    my_details = {}
    x = re.compile("\.|:|;|\s|&")
    edited_game_title= re.sub(x,"-", name).lower().replace("---","-").replace("--","-").replace("'", "")
    # periods at the end of the url will show up as a dash. This conditional is to get rid of it.
    if edited_game_title[-1] == "-":
        edited_game_title = edited_game_title[:-1]
        print(f"\nGAME TITLE: {edited_game_title}")
    else:
        print(f"\nGAME TITLE: {edited_game_title}")
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    my_details["Name"]=name
    
    html= browser.html
    my_soup = soup(html, "lxml")
    # Grabbing side details
    try:
        side_details = my_soup.find_all("div", class_="details side_details")[1]
        unordered_list = side_details.select("ul.summary_details li")
        try:
            developer = unordered_list[0].find("span", class_="data").get_text()
            my_details["developer"]=developer
        except:
            pass
        try:
            number_players = unordered_list[2].find("span",class_="data").get_text()
            my_details["number_players"]=number_players
        except:
            pass
        try:
            rating = unordered_list[4].find("span", class_="data").get_text()
            my_details["rating"]=rating
        except:
            pass
        game_side_details.append(my_details)
        print('SUCCESSFUL')
        print(f"This is the {counter}th game in the list.")
        counter+=1
    except:
        abberant_side_detail_games.append(name)
        print("SUCCESS but no side details")
        continue
        
            
        

        
        


GAME TITLE: wii-sports
SUCCESSFUL
This is the 1th game in the list.

GAME TITLE: mario-kart-wii
SUCCESSFUL
This is the 2th game in the list.

GAME TITLE: wii-sports-resort
SUCCESSFUL
This is the 3th game in the list.

GAME TITLE: new-super-mario-bros
SUCCESSFUL
This is the 4th game in the list.

GAME TITLE: wii-play
SUCCESSFUL
This is the 5th game in the list.

GAME TITLE: new-super-mario-bros-wii
SUCCESSFUL
This is the 6th game in the list.

GAME TITLE: nintendogs
SUCCESS but no side details

GAME TITLE: mario-kart-ds
SUCCESSFUL
This is the 7th game in the list.

GAME TITLE: wii-fit
SUCCESSFUL
This is the 8th game in the list.

GAME TITLE: wii-fit-plus
SUCCESSFUL
This is the 9th game in the list.

GAME TITLE: kinect-adventures!
SUCCESSFUL
This is the 10th game in the list.

GAME TITLE: grand-theft-auto-v
SUCCESSFUL
This is the 11th game in the list.

GAME TITLE: grand-theft-auto-san-andreas
SUCCESSFUL
This is the 12th game in the list.

GAME TITLE: brain-age-train-your-brain-in-minut

SUCCESSFUL
This is the 92th game in the list.

GAME TITLE: halo-3-odst
SUCCESSFUL
This is the 93th game in the list.

GAME TITLE: red-dead-redemption
SUCCESSFUL
This is the 94th game in the list.

GAME TITLE: super-mario-sunshine
SUCCESSFUL
This is the 95th game in the list.

GAME TITLE: driver
SUCCESSFUL
This is the 96th game in the list.

GAME TITLE: kinect-sports
SUCCESSFUL
This is the 97th game in the list.

GAME TITLE: gears-of-war-3
SUCCESSFUL
This is the 98th game in the list.

GAME TITLE: gears-of-war
SUCCESSFUL
This is the 99th game in the list.

GAME TITLE: metal-gear-solid-2-sons-of-liberty
SUCCESSFUL
This is the 100th game in the list.

GAME TITLE: metal-gear-solid-4-guns-of-the-patriots
SUCCESSFUL
This is the 101th game in the list.

GAME TITLE: metal-gear-solid
SUCCESSFUL
This is the 102th game in the list.

GAME TITLE: the-last-of-us
SUCCESSFUL
This is the 103th game in the list.

GAME TITLE: final-fantasy-xii
SUCCESSFUL
This is the 104th game in the list.

GAME TITLE: l

SUCCESSFUL
This is the 184th game in the list.

GAME TITLE: fable-ii
SUCCESSFUL
This is the 185th game in the list.

GAME TITLE: super-mario-3d-world
SUCCESSFUL
This is the 186th game in the list.

GAME TITLE: batman-arkham-asylum
SUCCESSFUL
This is the 187th game in the list.

GAME TITLE: assassins-creed-revelations
SUCCESSFUL
This is the 188th game in the list.

GAME TITLE: namco-museum
SUCCESSFUL
This is the 189th game in the list.

GAME TITLE: metal-gear-solid-3-snake-eater
SUCCESSFUL
This is the 190th game in the list.

GAME TITLE: daxter
SUCCESSFUL
This is the 191th game in the list.

GAME TITLE: fifa-soccer-06
SUCCESSFUL
This is the 192th game in the list.

GAME TITLE: eyetoy-play
SUCCESSFUL
This is the 193th game in the list.

GAME TITLE: uncharted-4-a-thiefs-end
SUCCESSFUL
This is the 194th game in the list.

GAME TITLE: gran-turismo-5-prologue
SUCCESSFUL
This is the 195th game in the list.

GAME TITLE: assassins-creed-revelations
SUCCESSFUL
This is the 196th game in the list.

In [21]:
abberant_side_detail_games

[{'Name': 'Kevin Van Dam: Big Bass Challenge'},
 {'Name': 'Glacier2'},
 {'Name': 'Memories Off 6: T-Wave'},
 {'Name': 'Bullet Girls 2'},
 {'Name': 'Sleepover Party'}]

In [ ]:
game_side_details = []
problem_games = []
abberant_side_detail_games = []
game_info_list = []
counter = 1

# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list[12000:15000]:
    game_scores = {}
    x = re.compile("\.|:|;|\s|&")
    edited_game_title= re.sub(x,"-", name).lower().replace("---","-").replace("--","-").replace("'", "")
    # periods at the end of the url will show up as a dash. This conditional is to get rid of it.
    if edited_game_title[-1] == "-":
        edited_game_title = edited_game_title[:-1]
        print(f"\nGAME TITLE: {edited_game_title}")
    else:
        print(f"\nGAME TITLE: {edited_game_title}")
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    game_scores["Name"]=name
    
    html= browser.html
    my_soup = soup(html, "lxml")
    # Grabbing the Metascore
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores["metascore"]= metascore
    except:
        problem_games.append(name)
        print('THIS IS A PROBLEM CHILD')
        continue
    # Grabbing the Userscore
    try:
        userscore_first = my_soup.find("div", class_="details side_details")
        user_score = userscore_first.select("div.metascore_w")[0].get_text()
        game_scores["user_score"]= user_score
    except:
        problem_games.append(name)
        print("THIS IS A PROBLEM CHILD")
        continue
    # Grabbing the release data
    try:
        product_data = my_soup.find("div",class_="product_data")
        release_data = product_data.find("li", class_="summary_detail release_data")
        release_date = release_data.find("span",class_="data").get_text()
        game_scores["release_date"]=release_date
    except:
        problem_games.append(name)
        print("THIS IS A PROBLEM CHILD")
        continue
    # Grabbing the positive, mixed, and negative critic review numbers
    try:
        critic_reviews = []
        positive_scores = my_soup.find("div",class_="distribution_wrap")
        x = positive_scores.select("ol.score_counts.hover_none span.count")
        for i in x:
            critic_reviews.append(i.get_text())
        game_scores["positive_critics"] = critic_reviews[0]
        game_scores["neutral_critics"]=critic_reviews[1]
        game_scores["negative_critics"]=critic_reviews[2]
    except:
        problem_games.append(name)
        print('THIS IS A PROBLEM CHILD')
        continue
    # Grabbing the positive, mixed, and negative user review numbers
    try:
        user_reviews = []
        user_scores = my_soup.find("div",class_="user_reviews_module")
        drilling_down = user_scores.find("div",class_="distribution_wrap")
        ordered_list_html = drilling_down.select("ol.score_counts.hover_none span.count")
        for i in ordered_list_html:
            user_reviews.append(i.get_text())
        game_scores["positive_users"] = user_reviews[0]
        game_scores["neutral_users"]=user_reviews[1]
        game_scores["negative_users"]=user_reviews[2]
    except:
        problem_games.append(name)
        print("THIS IS A PROBLEM CHILD")
        continue
        

In [20]:
game_side_details

[{'Name': 'Stolen',
  'developer': '\n                    Blue 52                ',
  'number_players': '1 Player',
  'rating': 'T'},
 {'Name': 'Monster High: New Ghoul in School',
  'developer': '\n                    Torus Games, Little Orbit                ',
  'number_players': 'On GameFAQs'},
 {'Name': 'Rugby Challenge 3',
  'developer': '\n                    Wicked Witch Software                ',
  'number_players': 'Up to 8 ',
  'rating': 'E'},
 {'Name': 'Superstars V8 Racing',
  'developer': '\n                    Milestone S.r.l                ',
  'number_players': 'On GameFAQs'},
 {'Name': 'Hatsune Miku: Project Diva X',
  'developer': '\n                    Sega                ',
  'number_players': 'No Online Multiplayer',
  'rating': 'T'},
 {'Name': 'Pro Race Driver',
  'developer': '\n                    Codemasters                ',
  'number_players': '1-4 ',
  'rating': 'T'}]

In [ ]:
#scraping_df = pd.DataFrame(game_info_list)
#scraping_df.to_csv("../Resources/scraped_ratings2")

In [ ]:
# get developer, genre, number_players, rating, release_date

In [ ]:
len(problem_games)

In [ ]:
#problem_ratings = pd.DataFrame(problem_games)
#problem_ratings.to_csv("../Resources/problem_ratings2")

In [ ]:
#'& III ! Pokémon

In [ ]:
len(abberant_side_detail_games)